In [1]:
import os
os.chdir("../function")

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import load_data
import Vaccine_DF
import multiprocessing
from mapping import generate_mapping_function
from mapping_dict import get_breed_dict

In [3]:
dataconn = load_data.LoadData()

ShowDB_Df = dataconn.select_data(
    "show databases"
    )
Schema_all = ShowDB_Df.iloc[0:-4,:].values.tolist()
Schema_all = sum(Schema_all, [])

# disconnect from server
dataconn.close()

In [4]:
data = None
start = time.time()
with multiprocessing.Pool(48)  as pool:
    data = list(pool.map(Vaccine_DF.makeDf, Schema_all))
end = time.time()
print(end - start)

989.vital doesn't exist

974.6807947158813


In [5]:
data = pd.concat(data)
data

,Name2,Name3,Sex,Diagnosis,Day,_Month,_Year
0,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4020.0,132.0,11.0
1,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4051.0,133.0,11.0
2,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4082.0,134.0,11.0
3,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4112.0,135.0,11.0
4,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4143.0,136.0,11.0
...,...,...,...,...,...,...,...
16136,Shiba Inu,시바 이누,3,"Heartgard(11~22kg,Green)",500.0,16.0,1.0
16137,Shiba Inu,시바 이누,3,외부기생충-Frontline(10~20kg),500.0,16.0,1.0
16138,Poodle,푸들,1,데피니트(4~10kg),2570.0,84.0,7.0
16139,Yorkshire Terrier,요크셔 테리어,4,광견병(Rabies),2387.0,78.0,6.0


In [6]:
data.isnull().sum()

Name2             0
Name3             0
Sex               0
Diagnosis         0
Day          162905
_Month       162905
_Year        162905
dtype: int64

In [7]:
data.dropna(inplace=True)

In [8]:
data = data.reset_index(drop=True)

In [9]:
data

,Name2,Name3,Sex,Diagnosis,Day,_Month,_Year
0,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4020.0,132.0,11.0
1,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4051.0,133.0,11.0
2,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4082.0,134.0,11.0
3,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4112.0,135.0,11.0
4,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4143.0,136.0,11.0
...,...,...,...,...,...,...,...
41317905,Shiba Inu,시바 이누,3,"Heartgard(11~22kg,Green)",500.0,16.0,1.0
41317906,Shiba Inu,시바 이누,3,외부기생충-Frontline(10~20kg),500.0,16.0,1.0
41317907,Poodle,푸들,1,데피니트(4~10kg),2570.0,84.0,7.0
41317908,Yorkshire Terrier,요크셔 테리어,4,광견병(Rabies),2387.0,78.0,6.0


In [10]:
mapping_dict = get_breed_dict()

tqdm.pandas()
data["Name"] = data.progress_apply(lambda row : generate_mapping_function(row,mapping_dict),axis=1)

100%|██████████| 41317910/41317910 [55:32<00:00, 12399.95it/s] 


In [11]:
data = data[(data["Sex"].isin([1,2,3,4])) & (data["Day"] > 0)]
data = data[data["Name"].isnull() == False]
data = data[data["Name"] != "Unknown"]
data

,Name2,Name3,Sex,Diagnosis,Day,_Month,_Year,Name
0,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4020.0,132.0,11.0,Maltese
1,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4051.0,133.0,11.0,Maltese
2,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4082.0,134.0,11.0,Maltese
3,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4112.0,135.0,11.0,Maltese
4,Maltese,몰티즈,2,"Heartgard(1~11kg,Blue)",4143.0,136.0,11.0,Maltese
...,...,...,...,...,...,...,...,...
41317904,Bichon Frise,비숑 프리제,2,기관지염백신(Kennel Cough),166.0,5.0,0.0,Bichon_Frise
41317905,Shiba Inu,시바 이누,3,"Heartgard(11~22kg,Green)",500.0,16.0,1.0,Shiba
41317906,Shiba Inu,시바 이누,3,외부기생충-Frontline(10~20kg),500.0,16.0,1.0,Shiba
41317908,Yorkshire Terrier,요크셔 테리어,4,광견병(Rabies),2387.0,78.0,6.0,Yorkshire Terrier


In [13]:
data["뿜빵이"] = "예방"
data.to_csv("../File/Vaccine.csv",index=False)